<a href="https://colab.research.google.com/github/antonin97/bc_thesis/blob/main/substitution_cipher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Google Colab environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/drive/My Drive/bc_crypto'

In [ ]:
!pip install unidecode

### Local environment

In [ ]:
my_path =  '.'

### Imports

In [ ]:
import numpy as np
import regex
from unidecode import unidecode
import math
import csv
import requests
import tensorflow as tf
import keras
import random
import pandas as pd
import copy
import os
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.lines import Line2D
from scipy.stats import chisquare
import pickle

### Loading text data
for testing and analysis purposes

In [ ]:
# simple test for helper functions testing
format_test_text = 'Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman?! sößáěř'

In [ ]:
# loading the testing data from Robinson Crusoe
with open(f'{my_path}/wikidata/testing_texts_robinson.txt', 'r') as file:
  content = file.read().replace('\n', '')

  # array of 50 texts of length 512
  testing_texts = [content[i:i+512] for i in range(0, 512*50, 512)]

# function to retrieve testing texts from Robinson Crusoe
def get_testing_texts(n=1):
  if n == 1:
    return random.sample(testing_texts, n)[0]
  return random.sample(testing_texts, n)

In [ ]:
# loading the whole book Peter Pan used for creating the n-gram matrix
file_path = f'{my_path}/wikidata/peter_pan.txt'
with open(file_path, 'r', encoding='utf-8') as file:
  petr_pan = file.read()


In [ ]:
# for creating the figures 4 & 5
texts_1024_10 = [testing_texts[i * 2] + testing_texts[i * 2 + 1] for i in range(10)]
texts_128_10 = [testing_texts[i][:128] for i in range(10)]

In [ ]:
# loading data from the wikipedia webscrapping
file_path = f'{my_path}/wikidata/512_data_encoded.csv'

# first column = string, encrypted text | second  column = float, level of encoding <0, 1>
column_defaults = [tf.string, tf.float32]

# create a CSV dataset
dataset = tf.data.experimental.CsvDataset(
  file_path,
  record_defaults=column_defaults,
  header=True,
)

### RNN import

In [ ]:
# custom n-gram split function for the model
def ngram_split(text):
  characters = tf.strings.unicode_split(text, 'UTF-8')
  # Create character n-grams
  return tf.strings.ngrams(
    characters,
    ngram_width=ngram_size, # global parameter
    separator=''  # join n-grams without spaces
  )
ngram_size = 3

In [ ]:
# model to be loaded (1-10)
model_nr = '10'

model_path = f'{my_path}/models/model_{model_nr}.tf'

# loading the model
rnn_model = tf.keras.models.load_model(model_path, custom_objects={
  'ngram_split': ngram_split
  })

In [ ]:
# defining the RNN function
def rnn_estimate_function(text):
  # turning the string into tf dataset
  dataset = tf.data.Dataset.from_tensor_slices(([text]))
  dataset = dataset.batch(1)
  rnn_estimate = rnn_model.predict(dataset, verbose=0)[0][0]
  return(rnn_estimate)

### Helper functions

In [ ]:
# formatting the text to contain only 27 characters A-Z and _ as whitespace
def format_text(input_text):

  # if text already formatted
  if regex.match(r'^[A-Z_]+$', input_text):
    return input_text

  # cleaning the text from all language-specific characters
  cleaned_text = unidecode(input_text)

  # remove non-alpha characters and transform the text into words connected by an underscore
  words = ''.join(char if char.isalpha() or char.isspace() else '' for char in cleaned_text).split()
  formatted_text = '_'.join(word.upper() for word in words)

  # sanity check
  assert regex.match(r'^[A-Z_]+$', formatted_text), 'contains forbidden characters'

  return formatted_text

In [ ]:
# formatting + vectorizing A-Z_ string --> (0-26) vector
def text_to_vector(text):
  text = format_text(text)
  # assigning ordinate (26 for the whitespace, underscore)
  char_to_number = {char: ord(char) - ord('A') if char.isalpha() else 26 for char in text}
  vector = np.array([char_to_number[char] for char in text])
  return vector

In [ ]:
# (0-26) vector --> A-Z_ string
def vector_to_text(vector):
  number_to_char = {i: chr(i + ord('A')) if i < 26 else '_' for i in range(27)}
  text = ''.join(number_to_char[number] for number in vector)
  return text

In [ ]:
# testing
print(vector_to_text(text_to_vector(format_test_text)))

### Substitute cipher functions

In [ ]:
# key is a permutation vector of numbers 0-26 (A-Z+_)
# plaintext can be inserted without formatting, will be formatted during text_to_vector call
def substitute_encrypt(plaintext, key):
  plaintext_vector = text_to_vector(plaintext)
  encrypted_vector = np.array([key[plaintext_vector[i]] for i in range(len(plaintext_vector))])
  encrypted_text = vector_to_text(encrypted_vector)
  return encrypted_text

In [ ]:
# same as encrypting, but creating an inversion of the key beforehand
def substitute_decrypt(ciphertext, key):
  inverse_key = np.array([np.where(key == i)[0][0] for i in range(27)])
  encrypted_vector = text_to_vector(ciphertext)
  plaintext_vector = np.array([inverse_key[encrypted_vector[i]] for i in range(len(encrypted_vector))])
  decrypted_text = vector_to_text(plaintext_vector)
  return decrypted_text

In [ ]:
# creating a permutation key, that will always yield the same percentage of vector elements to be switched up
# percent_mix - how many percent of the vector elements should be switched up
# n_elements - how many elements should the vector contain (0 - n-1)
def create_permutation_key(n_displace, n_elements=27):
    assert n_displace <= n_elements
    array = np.arange(n_elements)
    # ensuring no bug
    if n_displace == 1: n_displace = 0
    # randomly choose indices to displace
    indices = np.arange(n_elements)
    np.random.shuffle(indices)
    indices_to_displace = sorted(indices[:n_displace])

    # creating a copy to keep track of indices that can be still used
    indices_remaining = [i for i in indices_to_displace]

    for index in indices_to_displace:
        # flag for adding an element back to remaining
        add = False

        # if index hasn't been used for swap yet, it must be removed from the options
        if index in indices_remaining:
            indices_remaining.remove(index) # removing current index to ensure displacement
            add = True # flag for adding it back again after iteration

            # edge case - no more elements to swap the index with
            if len(indices_remaining) == 0:
                indices_to_displace.remove(index)
                # randomly swapping the last element with one of the others - certainly it won't happen, that displacement wouldn't hapen
                swap_index = np.random.choice(indices_to_displace)
                array[index] = array[swap_index]
                array[swap_index] = index
                break

        new_value = np.random.choice(indices_remaining) # choosing a value to insert to the given index
        array[index] = new_value # swapping the elements
        indices_remaining.remove(new_value) # removing the used index (can't be used twice)
        if add:
            indices_remaining.append(index) # adding back the index as value

    if type(array) == list:
      return array
    return array.tolist()



In [ ]:
# function that takes in a key and performs exactly n_change changes in it
def change_permutation_key(n_change, key):
  key = list(key)
  indices = np.arange(len(key))
  np.random.shuffle(indices)
  indices_to_displace = sorted(indices[:n_change])
  for i in range(len(indices_to_displace) - 1):
    key[indices_to_displace[i % n_change]], key[indices_to_displace[(i + 1) % n_change]] = key[indices_to_displace[(i + 1) % n_change]], key[indices_to_displace[i % n_change]]
  return key

In [ ]:
# comparing two texts of the same length, returning percentage of same placed characters
def compare_texts(text1, text2):
  assert len(text1) == len(text2)
  return sum([text1[i] == text2[i] for i in range(len(text1))]) / len(text1)

In [ ]:
# local search for a key to substitution cipher
# ciphertext - encrypted using unknown key
# n_iterations - iterations of the Monte Carlo algorithm
# likelihood_func - either markov chains analysis likelihood or the RNN estimate
# original_text - the original plaintext, for analysis purposes
# key - if a predefined key should be used, else a random key
# return_key - if the final key should be returned, else analysis information is returned
def monte_carlo_algorithm(ciphertext, n_iterations, likelihood_func, original_text=None, key=None, return_key=False):

  # array for storing algorithm data
  confusion_matrix_data = []
  likelihoods = []
  true_percentages = []

  # creating a random key - vector of numbers 0 - 26
  if not np.any(key):
    key = np.random.permutation(27)
  # using key passed as a parameter (e.g. fixed points of permutation simulation)
  else:
    key = np.array(key)

  # initial evaluation values
  iter_decrypt = substitute_decrypt(ciphertext, key)
  iter_likelihood = likelihood_func(iter_decrypt)
  likelihoods.append(iter_likelihood)

  # if plaintext is passed as an argument, we can observe the true percentage
  if original_text:
    true_percentage = compare_texts(iter_decrypt, original_text)
    true_percentages.append(true_percentage)

  # tracking made actions
  swaps, rejected, randomwalk = 0,0,0

  for _ in range(n_iterations):
    new_key = np.copy(key)

    # randomly swapping two elements of the key vector
    index1, index2 = np.random.choice(len(new_key), size=2, replace=False)
    new_key[index1], new_key[index2] = new_key[index2], new_key[index1]

    # decrypting text with the new vector and computing its likelihood
    new_iter_decrypt = substitute_decrypt(ciphertext, new_key)
    new_iter_likelihood = likelihood_func(new_iter_decrypt)

    if original_text:
      new_true_percentage = compare_texts(new_iter_decrypt, original_text)
      # boolean --> has the true likelihood actually improved?
      true_q = new_true_percentage > true_percentage

    # computing ratio --> how much has the internal likelihood improved?
    q = new_iter_likelihood / iter_likelihood

    # if the internal likelihood improved, swap is approved --> hill climb
    if (q > 1):
      key = new_key
      iter_decrypt = new_iter_decrypt
      iter_likelihood = new_iter_likelihood
      swaps += 1

      if original_text:
        true_percentage = new_true_percentage
        # computing data for confussion matrix
        # 3 == True Positive
        if true_q:
          confusion_matrix_data.append(3)
        # 1 == False Positive
        else:
          confusion_matrix_data.append(1)

    # if the internal likelihood has only slightly decreased + odds --> random walk
    elif (q > 0.9 and np.random.rand() < 0.001):
      key = new_key
      iter_decrypt = new_iter_decrypt
      iter_likelihood = new_iter_likelihood
      randomwalk += 1

      if original_text:
        true_percentage = new_true_percentage
        # computing data for confussion matrix
        # 3 == True Positive
        if true_q:
          confusion_matrix_data.append(3)
        # 1 == False Positive
        else:
          confusion_matrix_data.append(1)

    # if the internal likelihood hasn't improved --> reject
    else:
      rejected += 1

    if original_text:
      # computing data for confussion matrix
      # 2 == True Negative
      if not true_q:
        confusion_matrix_data.append(2)
      # 0 == False Negative
      else:
        confusion_matrix_data.append(0)

    likelihoods.append(iter_likelihood)
    if original_text:
      true_percentages.append(true_percentage)

    # console log
    if (_ % 1000 == 0):
        print(f'{_}. iteration', iter_likelihood)

  print(f'swaps/rejected/random: {swaps}/{rejected}/{randomwalk}')
  if return_key:
    return key
  return likelihoods, true_percentages, confusion_matrix_data

### Breaking Caesar's cipher (Figure 2)

In [ ]:
# obtained from the web Letter Frequencies in the English Language
alphabet_freq = {
  'E' :	0.111607,
  'M' :	0.030129,
  'A' :	0.084966,
  'H' :	0.030034,
  'R' :	0.075809,
  'G' :	0.024705,
  'I' :	0.075448,
  'B' :	0.020720,
  'O' :	0.071635,
  'F' :	0.018121,
  'T' :	0.069509,
  'Y' :	0.017779,
  'N' :	0.066544,
  'W' :	0.012898,
  'S' :	0.057351,
  'K' :	0.011016,
  'L' :	0.054893,
  'V' :	0.010074,
  'C' :	0.045388,
  'X' :	0.002902,
  'U' :	0.036308,
  'Z' :	0.002722,
  'D' :	0.033844,
  'J' :	0.001965,
  'P' :	0.031671,
  'Q' :	0.001962,
}

In [ ]:
# Apply Caesar cipher to the text with a given shift
# expects texts with only A-Z letters, no whitespaces
def caesar_shift(text, shift):
  shifted_text = ''
  for char in text.upper():
    if 'A' <= char <= 'Z':
      shifted_text += chr((ord(char) - ord('A') + shift) % 26 + ord('A'))
    else:
      shifted_text += char
  return shifted_text

# calculates frequencies for all letters in the alphabet for a given text
def letter_frequency(text, alphabet_frequency):
    text = text.upper()  # normalize the input to uppercase
    letter_counts = {chr(i): 0 for i in range(ord('A'), ord('Z') + 1)}
    total_letters = 0
    for char in text:
      if 'A' <= char <= 'Z':
        letter_counts[char] += 1
        total_letters += 1

    # calculate frequencies and store into the dictionary
    if total_letters > 0:
      return {letter: count / total_letters for letter, count in letter_counts.items()}
    else:
      return {letter: 0 for letter in letter_counts}

# plot function for the Figure 2
def plot_letter_frequencies(text, alphabet_frequency):
  plt.figure(figsize=(15, 30))  # Adjusted figure size for 3x9 grid
  letters = sorted(alphabet_frequency.keys())

  axs = []  # list to store axes to access later for title update
  chi2_values = []  # store chi-squared statistics for identifying the minimal value

  # generate plots for all 26 shifts
  for shift in range(26):
    shifted_text = caesar_shift(text, shift)
    letter_freqs = letter_frequency(shifted_text, alphabet_frequency)

    # calculate Chi-Squared test
    observed = [letter_freqs.get(letter, 0) * len(shifted_text) for letter in letters]
    expected = [alphabet_frequency[letter] * len(shifted_text) for letter in letters]
    chi2_stat, p_val = chisquare(f_obs=observed, f_exp=expected)
    chi2_values.append(chi2_stat)

    # create a subplot for this shift in a 3x9 grid
    ax = plt.subplot(9, 3, shift + 1)
    axs.append(ax)

    alphabet_freqs = [alphabet_frequency[letter] for letter in letters]
    text_freqs = [letter_freqs.get(letter, 0) for letter in letters]

    # plot alphabet frequencies as bars and text frequencies as a line
    ax.bar(letters, alphabet_freqs, color='blue', alpha=0.7)
    ax.plot(letters, text_freqs, 'r--')
    ax.set_title(f'Shift {shift} | χ² = {chi2_stat:.3f}')
    ax.set_ylim(0, max(alphabet_freqs) + 0.05)  # adjust ylim to better fit the titles

  # identify the subplot with minimal Chi-Squared value
  min_chi2_index = chi2_values.index(min(chi2_values))
  axs[min_chi2_index].set_title(f'Shift {min_chi2_index} | χ² = {chi2_values[min_chi2_index]:.3f}', color='red')

  # use the last subplot to place the legend
  ax = plt.subplot(9, 3, 27)
  ax.legend(handles=[plt.Rectangle((0,0),1,1, color='blue', alpha=0.7), plt.Line2D([0],[0], color='red', linestyle='--')],
            labels=['English letters Frequency', 'Text letters Frequency'], loc='center')
  ax.axis('off')

  plt.tight_layout()
  plt.savefig(f'{my_path}/plots/caesar.png')
  plt.show()

#### Plot used in Figure 2

In [ ]:
text_example = 'The only thing that interferes with my learning is my education'
shift_key = tuple([i for i in range(6,27)]+[i for i in range(0,6)])
text_example = substitute_encrypt(text_example, shift_key)
plot_letter_frequencies(text_example, alphabet_freq)

### Markov Chains transitions

In [ ]:
# calculating digram matrix based on text
# only for alphabet of length 27 (A-Z + _)
def digram_matrix(text, logsafe=False):
  # logsafe - robust for the calculation (language analysis)
  if logsafe:
    matrix = np.ones((27, 27))
  # not logsafe - more precise
  else:
    matrix = np.zeros((27, 27))
  i = 0
  while i + 1 < len(text):
    ith = 26 if text[i] == '_' else (ord(text[i]) - ord('A'))
    jth = 26 if text[i + 1] == '_' else (ord(text[i + 1]) - ord('A'))
    matrix[ith][jth] += 1
    i += 1
  return matrix

In [ ]:
# creating digram transition matrix for the English language
# based on Petr Pan book
petr_pan = format_text(petr_pan)
TM_en = digram_matrix(petr_pan, logsafe=True)

In [ ]:
# calculating likelihood from digram matrix of English language and digram matrix of the ciphertext
def markov_chain_analysis(text):
  likelihood = np.sum(np.log(TM_en) * digram_matrix(text))
  return likelihood

### Comparing the hill climb with the Monte Carlo algorithm (Figure 4, 5)


In [ ]:
# local search for a key to substitution cipher - hill climb method
# ciphertext - encrypted using unknown key
# n_iterations - iterations of the hill climb algorithm
# likelihood_func - either markov chains analysis likelihood or the RNN estimate
def hill_climb_plaus(ciphertext, n_iterations, likelihood_func, hill=True):
  # np.arange to ensure the initial key is a derangement of the true key (worst case scenario)
  key = np.arange(27)


  iter_decrypt = substitute_decrypt(ciphertext, key)
  # initial likelihood
  iter_likelihood = likelihood_func(iter_decrypt)
  likelihoods = [iter_likelihood]

  if hill:
    for _ in range(n_iterations):
      top_swap = (0, 0)
      top_plaus = likelihoods[-1]
      # 27 * 26 / 2 = 351 běží loop
      for index1 in range(0, len(key)):
        for index2 in range(index1 + 1, len(key)):
          new_key = np.copy(key)
          new_key[index1], new_key[index2] = new_key[index2], new_key[index1]
          new_iter_decrypt = substitute_decrypt(ciphertext, new_key)
          new_iter_likelihood = likelihood_func(new_iter_decrypt)
          if new_iter_likelihood > top_plaus:
            top_plaus = new_iter_likelihood
            top_swap = (index1, index2)
      # if no change was done
      if top_plaus == likelihoods[-1]:
        likelihoods = likelihoods + [likelihoods[-1] for i in range(n_iterations - len(likelihoods))]
        return likelihoods

      key[top_swap[0]], key[top_swap[1]] = key[top_swap[1]], key[top_swap[0]]

      likelihoods.append(top_plaus)
    return likelihoods


  else:
    for _ in range(n_iterations * 351):
      new_key = np.copy(key)

      # randomly swapping two elements of the key vector
      index1, index2 = np.random.choice(len(new_key), size=2, replace=False)
      new_key[index1], new_key[index2] = new_key[index2], new_key[index1]

      # decrypting text with the new vector and computing its likelihood
      new_iter_decrypt = substitute_decrypt(ciphertext, new_key)
      new_iter_likelihood = likelihood_func(new_iter_decrypt)

      q = new_iter_likelihood / iter_likelihood

      if (q > 1):
        key = new_key
        iter_decrypt = new_iter_decrypt
        iter_likelihood = new_iter_likelihood
      elif (q > 0.95 and np.random.rand() < 0.001):
        key = new_key
        iter_decrypt = new_iter_decrypt
        iter_likelihood = new_iter_likelihood

      likelihoods.append(iter_likelihood)
    return likelihoods

In [ ]:
# Figure 4
text = random.sample(texts_1024_10, 1)[0]

# Figure 5
#text = random.sample(texts_128_10, 1)[0]

plaintext_likelihood = markov_chain_analysis(text)

likelihoods_hillclimb = []
# number of different initial keys
for _ in range(5):
  # derangement key for the decryption
  key = create_permutation_key(27, 27)
  ciphertext = substitute_encrypt(format_text(text), key)
  likelihoods_hillclimb.append(hill_climb_plaus(ciphertext, 33, markov_chain_analysis))

likelihoods_monte_carlo = []
# number of different initial keys
for _ in range(5):
  # derangement key for the decryption
  key = create_permutation_key(27, 27)
  ciphertext = substitute_encrypt(format_text(text), key)
  likelihoods_monte_carlo.append(hill_climb_plaus(ciphertext, 33, markov_chain_analysis, hill=False))

# multiplying by the number of swaps
likelihoods_hillclimb_final = []
for i in likelihoods_hillclimb:
  x = []
  for j in i:
    for _ in range(351):
      x.append(j)
  likelihoods_hillclimb_final.append(x)

#### Plot used in Figures 4 and 5

In [ ]:
# get colormaps for green and blue shades
green_cmap = get_cmap('Greens')
blue_cmap = get_cmap('Blues')

# determine the number of lines to plot
num_hillclimb = len(likelihoods_hillclimb_final)
num_monte_carlo = len(likelihoods_monte_carlo)

# generate color shades from 40% to 100% intensity
green_colors = [green_cmap(0.4 + 0.6 * i / (num_hillclimb - 1)) for i in range(num_hillclimb)]
blue_colors = [blue_cmap(0.4 + 0.6 * i / (num_monte_carlo - 1)) for i in range(num_monte_carlo)]

plt.figure(figsize=(10, 6))

# plot hill climb data with shades of green
for i, color in zip(likelihoods_hillclimb_final, green_colors):
    plt.plot(i, linestyle='-', color=color)

# plot monte carlo data with shades of blue
for j, color in zip(likelihoods_monte_carlo, blue_colors):

    plt.plot(j, linestyle='-', color=color)

# plot plaintext likelihood line
plt.plot([plaintext_likelihood for _ in range(33 * 351)], linestyle='-', color='red')

# create custom legend
custom_lines = [Line2D([0], [0], color=green_cmap(0.7), lw=2),
                Line2D([0], [0], color=blue_cmap(0.7), lw=2),
                Line2D([0], [0], color='red', lw=2)]

plt.legend(custom_lines, ['Hill Climb method', 'Monte Carlo method', 'Plaintext log-likelihood'])

plt.xlabel('iteration')
plt.ylabel('log-likelihood')
plt.grid(True)
plt.savefig(f'{my_path}/plots/likelihoods_montecarlo_hillclimb.png')
plt.show()

### Percentage of correctly placed characters (Table 3)

#### Plot used in Table 3

In [ ]:
walt_disney_text = 'THEWAYTOGETSTARTEDISTOQUITTALKINGANDBEGINDOING'
for i in [0, 5, 10, 15, 20, 26]:
  key = create_permutation_key(i, 26)
  print(substitute_encrypt(walt_disney_text, key), round(1 - (i / 26), 2) )

### Comparison of % of correctly placed characters and models estimates (Figure 14, 15, 18, 21, 22, 23)

In [ ]:
total_items = 250000 # hardcoded
text_length = 512

full_dataset = dataset.shuffle(buffer_size=total_items) # to ensure uniform distribution, we need the buffer to consist of all the data

# filtering only plaintext data - no encryption
def filter_plaintext(data, label):
    return tf.equal(label, 1.0)

# Create plaintext_dataset by filtering full_dataset
plaintext_dataset = full_dataset.filter(filter_plaintext).batch(50)


In [ ]:
true = []
predicted = []

# for each magnitude of derangement (27) one batch (50) is tested
for i in range(1, 28):
  for text, _ in plaintext_dataset.take(1):
    for datapoint in text:
      datapoint = datapoint.numpy().decode('utf-8')
      key = create_permutation_key(i, n_elements=27)
      ciphertext = substitute_encrypt(datapoint, key)
      true.append(compare_texts(datapoint, ciphertext))
      predicted.append(rnn_estimate_function(ciphertext))

sorted_true, sorted_predicted = zip(*sorted(zip(true, predicted), reverse=True))
sorted_true = list(sorted_true)
sorted_predicted = list(sorted_predicted)

#### Plot used in Figures 14, 15, 18, 21, 22, 23

In [ ]:
plt.scatter(range(len(sorted_predicted)), sorted_predicted, color='blue', label='Predicted', alpha=0.3)

# create a line plot for the true data
plt.plot(range(len(sorted_true)), sorted_true, color='red', label='True')

# customize the plot
plt.ylabel('likelihood')
plt.xlabel('Index')
plt.title('True vs Predicted likelihood')
plt.legend()

# show the plot
plt.savefig(f'{my_path}/plots/likelihoods_{model_nr}.png')
plt.show()

### Histogram RNN estimates of derangements (Figure 16)

In [ ]:
# true & predicted values
true = []
guessed = []
for i in range(40):
  for text, _ in plaintext_dataset.take(1):
    for datapoint in text:
      datapoint = datapoint.numpy().decode('utf-8')
      key = create_permutation_key(27, n_elements=27)
      ciphertext = substitute_encrypt(datapoint, key)
      true.append(compare_texts(datapoint, ciphertext))
      guessed.append(rnn_estimate_function(ciphertext))

#### Plot used in Figure 16

In [ ]:
plt.hist(guessed, bins=10, range=(0, 1), edgecolor='black')

# adding title and labels
plt.title('Histogram of Data')
plt.xlabel('likelihood')
plt.ylabel('Frequency')

# display the histogram
plt.savefig(f'{my_path}/plots/likelihoods_histogram.png')
plt.show()

### Monte Carlo algorithm results with RNN estimates (Table 5)

#### Used in Table 5

In [ ]:
text = 'IN_THE_HEART_OF_THE_BUSTLING_CITY_WHERE_SKYSCRAPERS_REACH_FOR_THE_CLOUDS_A_HIDDEN_OASIS_BLOOMS_A_QUAINT_PARK_ADORNED_WITH_VIBRAN'
for i in range(10):
  key = create_permutation_key(27, 27)
  ciphertext = substitute_encrypt(text, key)
  found_key = monte_carlo_algorithm(ciphertext, 2001, rnn_estimate_function, return_key=True)
  decrypted = substitute_decrypt(ciphertext, found_key)
  print(decrypted[:50] + '...')
  print('likelihood: ', rnn_estimate_function(decrypted))
  correct = sum([1 if decrypted[i] == text[i] else 0 for i in range(len(ciphertext))]) / len(ciphertext)
  print('Correct %:', correct)


### Compare true percentage of characters and the RNN estimate (Figure 17, 19)

#### Plot used in Figures 17 and 19

In [ ]:
text = get_testing_texts(1)
key = create_permutation_key(27, 27)
ciphertext = substitute_encrypt(text, key)

likelihoods, true_percentages, _ = monte_carlo_algorithm(ciphertext, 3000, rnn_estimate_function, original_text=text)

plt.figure(figsize=(10, 6))

plt.plot(likelihoods, label='RNN estimate', color='blue')
plt.plot(true_percentages, label='% correct characters', color='orange')

# add labels and title
plt.xlabel('iteration')
plt.ylabel('percentage')
plt.legend()
plt.show()

### MSE vs. MAE (Figure 20)

#### Plot used in Figure 20

In [ ]:
# Define the range and the functions
x = np.linspace(0, 1, 100)
y1 = x**2
y2 = x

# Create the plot
plt.figure(figsize=(8, 6))
plt.plot(x, y1, label='MSE')
plt.plot(x, y2, label='MAE')
plt.xlabel('difference')
plt.ylabel('loss')
plt.legend()
plt.grid(True)
plt.show()

### RNN starting with a permutation key with multiple fixed points (Figure 24)

#### Plot used in Figure 24

In [ ]:
# how many fixed points should the keys have
n_fixed_points = 15

text = get_testing_texts(1)
key = create_permutation_key(27, 27)

# change the original key...
intermediate_key = change_permutation_key(27 - n_fixed_points, key)
ciphertext = substitute_encrypt(text, key)

#... and pass it as a parameter
likelihoods, true_percentages, _ = monte_carlo_algorithm(ciphertext, 3001, rnn_estimate_function, key=intermediate_key, original_text=text)

plt.figure(figsize=(10, 6))

plt.plot(likelihoods, label='RNN estimate', color='blue')
plt.plot(true_percentages, label='% correct characters', color='orange')

plt.xlabel('iteration')
plt.ylabel('percentage')
plt.legend()
plt.show()

### Confusion matrices calculations (Table 6, 7, 8, 9)

#### Data used for Table 6 and 7
with a derangement key

In [ ]:
# getting 30 random texts
plaintext_texts = get_testing_texts(30)
#likelihood_func = rnn_estimate_function
likelihood_func = markov_chain_analysis

# data for the confusion matrix
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0

for plaintext_text in plaintext_texts:
  for i in range(10):
    # key to encrypt + create ciphertext
    encrypt_key = create_permutation_key(27, 27)
    ciphertext = substitute_encrypt(plaintext_text, encrypt_key)

    # ensuring derangement with 0 fixed points as the key to encrypt
    while True:
      key = create_permutation_key(27, 27)
      if not sum([key[i] == encrypt_key[i] for i in range(len(key))]):
        key = np.array(key)
        break

    # decrypting using the new key
    decrypted = substitute_decrypt(ciphertext, key)
    # internal likelihood (should be 0)
    original_plaus = likelihood_func(decrypted)

    # trying all possible swaps
    for index1 in range(27):
      for index2 in range(index1 + 1, 27):
        new_key = np.array(key)
        new_key[index1], new_key[index2] = new_key[index2], new_key[index1]

        correct_chars_before = compare_texts(decrypted, plaintext_text)
        decrypted_text = substitute_decrypt(ciphertext, new_key)
        new_correct_chars = compare_texts(decrypted_text, plaintext_text)
        mc_plaus = likelihood_func(decrypted_text)
        # true likelihood improved...
        if new_correct_chars > correct_chars_before:
          # ... and the model noticed --> TP
          if mc_plaus > original_plaus:
            true_pos += 1
          # ... and the model did NOT notice --> FN
          else:
            false_neg += 1
        # true likelihood did not improve...
        else:
          # ... and the model noticed --> TN
          if mc_plaus < original_plaus:
            true_neg += 1
          # ... and the model did NOT notice --> FP
          else:
            false_pos += 1

print(f'TP {true_pos}\nFP {false_pos}\nFN {false_neg}\nTN {true_neg}')

#### Data used for Table 8 and 9
with a key after 1000 iterations

In [ ]:
# getting 30 random texts
plaintext_texts = get_testing_texts(30)

#likelihood_func = rnn_estimate_function
likelihood_func = markov_chain_analysis

# data for the confusion matrix
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0

for plaintext_text in plaintext_texts:
  encrypt_key = create_permutation_key(27, 27)
  ciphertext = substitute_encrypt(plaintext_text, encrypt_key)


  for j in range(10):
    decrypt_key = create_permutation_key(27, 27)
    intermediate_key = monte_carlo_algorithm(ciphertext, 1001, likelihood_func, plaintext_text, return_key=True)
    current_decrypted_text = substitute_decrypt(ciphertext, intermediate_key)
    original_plaus = likelihood_func(current_decrypted_text)


    for index1 in range(27):
      for index2 in range(index1 + 1, 27):
        new_key = np.array(intermediate_key)
        new_key[index1], new_key[index2] = new_key[index2], new_key[index1]
        correct_chars_before = sum([1 if current_decrypted_text[j] == plaintext_text[j] else 0 for j in range(512)])
        new_decrypted_text = substitute_decrypt(ciphertext, new_key)
        new_correct_chars = sum([1 if plaintext_text[j] == new_decrypted_text[j] else 0 for j in range(512)])

        mc_plaus = likelihood_func(new_decrypted_text)

        # true likelihood improved...
        if new_correct_chars > correct_chars_before:
          # ... and the model noticed --> TP
          if mc_plaus > original_plaus:
            true_pos += 1
          # ... and the model did NOT notice --> FN
          else:
            false_neg += 1
        # true likelihood did not improve...
        else:
          # ... and the model noticed --> TN
          if mc_plaus < original_plaus:
            true_neg += 1
          # ... and the model did NOT notice --> FP
          else:
            false_pos += 1
print(f'TP {true_pos}\nFP {false_pos}\nFN {false_neg}\nTN {true_neg}\n\n')

### Sensitivity and Specificity in time (Figure 25, 26)

In [ ]:
# local search for a key to substitution cipher with checking several swaps each iterations
def monte_carlo_algorithm_sens_spec(ciphertext, n_iterations, likelihood_func, original_text, return_key=False):
  # array for storing algorithm data
  confusion_matrix_data = [{'TN':0, 'TP':0, 'FN':0, 'FP':0} for j in range(n_iterations // 200)]
  likelihoods = []
  true_percentages = []
  # creating a random key - vector of numbers 0 - 26
  key = np.random.permutation(27)


  # initial evaluation values
  iter_decrypt = substitute_decrypt(ciphertext, key)
  iter_likelihood = likelihood_func(iter_decrypt)
  true_percentage = compare_texts(iter_decrypt, original_text)

  # tracking made actions
  swaps, rejected, randomwalk = 0,0,0

  for i in range(n_iterations):
      new_key = np.copy(key)

      # after each iteration, simulation of 80 possible swaps is done
      #_______________________________________________________________________________________________________

      true_pos = 0
      true_neg = 0
      false_pos = 0
      false_neg = 0

      for swap in range(80):
          x_index1, x_index2 = random.sample(range(27), 2)
          x_original_plaus = iter_likelihood

          x_correct_chars_before = sum([1 if iter_decrypt[j] == original_text[j] else 0 for j in range(512)])

          x_new_key = np.array(key)
          x_new_key[x_index1], x_new_key[x_index2] = x_new_key[x_index2], x_new_key[x_index1]

          x_decrypted_text = substitute_decrypt(ciphertext, x_new_key)

          x_new_correct_chars = sum([1 if original_text[j] == x_decrypted_text[j] else 0 for j in range(512)])

          x_mc_plaus = likelihood_func(x_decrypted_text)

          # true likelihood improved...
          if x_new_correct_chars > x_correct_chars_before:
            # ... and the model noticed --> TP
            if x_mc_plaus > x_original_plaus:
              true_pos += 1
            # ... and the model did NOT notice --> FN
            else:
              false_neg += 1
          # true likelihood did not improve...
          else:
            # ... and the model noticed --> TN
            if x_mc_plaus < x_original_plaus:
              true_neg += 1
            # ... and the model did NOT notice --> FP
            else:
              false_pos += 1

      confusion_matrix_data[i // 200]['TP'] += true_pos
      confusion_matrix_data[i // 200]['TN'] += true_neg
      confusion_matrix_data[i // 200]['FP'] += false_pos
      confusion_matrix_data[i // 200]['FN'] += false_neg

      #_______________________________________________________________________________________________________


      # randomly swapping two elements of the key vector
      index1, index2 = np.random.choice(len(new_key), size=2, replace=False)
      new_key[index1], new_key[index2] = new_key[index2], new_key[index1]

      # decrypting text with the new vector and computing its likelihood
      new_iter_decrypt = substitute_decrypt(ciphertext, new_key)
      new_iter_likelihood = likelihood_func(new_iter_decrypt)
      new_true_percentage = compare_texts(new_iter_decrypt, original_text)

      # computing ratio --> how much has the internal likelihood improved?
      q = new_iter_likelihood / iter_likelihood

      # boolean --> has the true likelihood actually improved?
      true_q = new_true_percentage > true_percentage


      # if the internal likelihood improved, swap is approved --> hill climb
      if (q > 1):
          key = new_key
          iter_decrypt = new_iter_decrypt
          iter_likelihood = new_iter_likelihood
          true_percentage = new_true_percentage
          swaps += 1


      # if the internal likelihood has only slightly decreased + odds --> random walk
      elif (q > 0.9 and np.random.rand() < 0.001):
          key = new_key
          iter_decrypt = new_iter_decrypt
          iter_likelihood = new_iter_likelihood
          true_percentage = new_true_percentage
          randomwalk += 1

      # if the internal likelihood hasn't improved --> reject
      else:
          rejected += 1

      likelihoods.append(iter_likelihood)
      true_percentages.append(true_percentage)

      # console log
      if (i % 1000 == 0):
          print(f'{i}. iteration', iter_likelihood)
  print(f'swaps/rejected/random {swaps}/{rejected}/{randomwalk}')
  if return_key:
    return key
  return likelihoods, true_percentages, confusion_matrix_data

In [ ]:
# data_path = f'{my_path}/variables/saved_variables_rnn.pkl'
data_path = f'{my_path}/variables/saved_variables_mc.pkl'

# likelihood_function = rnn_estimate_function
likelihood_function = markov_chain_analysis

texts = get_testing_texts(30)
function_data = []

for text in texts:
  datapoint = {'likelihoods':None, 'true_percentages':None, 'sensitivity':None, 'specificity':None}
  key = np.random.permutation(27)
  ciphertext = substitute_encrypt(text, key)
  likelihoods, true_percentages, confusion_matrix_data = monte_carlo_algorithm_sens_spec(ciphertext, 2000, likelihood_function, text)
  sensitivity = []
  specificity = []

  # calculation sensitivity and specificity
  for i in confusion_matrix_data:
    sensitivity.append(i['TP'] / (i['TP'] + i['FN']))
    specificity.append(i['TN'] / (i['TN'] + i['FP']))

    # scaling Markov Chains log_likelihoods
    if likelihood_function == markov_chain_analysis:
      maxi = markov_chain_analysis(text)
      mini = min(likelihoods)
      likelihoods = [(i - mini) / (maxi - mini)  for i in likelihoods]

  datapoint['likelihoods'] = likelihoods
  datapoint['true_percentages'] = true_percentages
  datapoint['sensitivity'] = sensitivity
  datapoint['specificity'] = specificity

  # saving data each iteration
  function_data.append(datapoint)
  with open(data_path, 'wb') as file:
    pickle.dump({
        'mc_data': function_data},
                file)

  print('Done')


In [ ]:
def plot_time_conf_data(dictionary, RNN=False):
  log_likelihoods = dictionary['likelihoods']
  true_correct_chars = dictionary['true_percentages']
  sensitivity = dictionary['sensitivity']
  specificity = dictionary['specificity']

  # Creating x values for the lines
  x_values = np.arange(2000)

  # Creating x values for the bars
  bar_intervals = [(i * 200, (i + 1) * 200) for i in range(10)]

  fig, ax1 = plt.subplots(figsize=(14, 8))

  # Plotting log-likelihoods and true percentage of correct characters as lines

  if RNN:
    label_func = 'RNN estimate'
  else:
    label_func = 'scaled Log-Likelihood'

  ax1.plot(x_values, log_likelihoods, label=label_func, color='blue')
  ax1.plot(x_values, true_correct_chars, label='True % Correct Characters', color='green')
  ax1.set_xlabel('iteration')
  ax1.set_ylabel('percentage')
  ax1.legend(loc='upper right')

  # Plotting sensitivity and specificity as bars
  for i, (start, end) in enumerate(bar_intervals):
      ax1.bar(start + 50, sensitivity[i], width=100, color='red', alpha=0.5, label='Sensitivity' if i == 0 else '')
      ax1.bar(start + 150, specificity[i], width=100, color='orange', alpha=0.5, label='Specificity' if i == 0 else '')

  ax1.legend(loc='center right')

  plt.show()


#### The execution time is extreme, data can be loaded from drive

In [ ]:
# load the data
data_path = f'{my_path}/variables/saved_variables_rnn.pkl'
with open(data_path, 'rb') as file:
    data = pickle.load(file)

rnn_data = data['rnn_data']

In [ ]:
# load the data
data_path = f'{my_path}/variables/saved_variables_mc.pkl'
with open(data_path, 'rb') as file:
    data = pickle.load(file)

markov_chain_data = data['markov_chain_data']

#### Computing averages

In [ ]:
markov_chain_avg = {'likelihoods':np.zeros(2000), 'true_percentages':np.zeros(2000), 'sensitivity':np.zeros(10), 'specificity':np.zeros(10)}
n_data = len(markov_chain_data)
for mc in markov_chain_data:
  markov_chain_avg['likelihoods'] += np.array(mc['plausibilities']) / n_data
  markov_chain_avg['true_percentages'] += np.array(mc['true_plausibilities']) / n_data
  markov_chain_avg['sensitivity'] += np.array(mc['sensitivity']) / n_data
  markov_chain_avg['specificity'] += np.array(mc['specificity']) / n_data

In [ ]:
rnn_avg = {'likelihoods':np.zeros(2000), 'true_percentages':np.zeros(2000), 'sensitivity':np.zeros(10), 'specificity':np.zeros(10)}
n_data = len(rnn_data)
for rnn in rnn_data:
  rnn_avg['likelihoods'] += np.array(rnn['plausibilities']) / n_data
  rnn_avg['true_percentages'] += np.array(rnn['true_plausibilities']) / n_data
  rnn_avg['sensitivity'] += np.array(rnn['sensitivity']) / n_data
  rnn_avg['specificity'] += np.array(rnn['specificity']) / n_data

#### Plot used in Figure 25 and 26

In [ ]:
plot_time_conf_data(rnn_avg, RNN=True)

In [ ]:
plot_time_conf_data(markov_chain_avg)

### Confusion matrix decisions together with RNN estimates/log-likelihood (Figure 27, 28)

#### Plot used in Figure 27

In [ ]:
for _ in range(5):
  text = get_testing_texts(1)
  key = create_permutation_key(27, 27)

  plaintext_likelihood = markov_chain_analysis(text)

  ciphertext = substitute_encrypt(text, key)
  likelihoods, true_chars_percent, confus=monte_carlo_algorithm(ciphertext, 5001, markov_chain_analysis, original_text=text)

  plt.figure(figsize=(10, 6))

  likelihoods = [(i - min(likelihoods)) / (plaintext_likelihood - min(likelihoods)) for i in likelihoods]

  # plot the likelihood and correct characters % using the index as the x-value
  plt.plot(likelihoods, label='scaled log-likelihood', color='blue')
  plt.plot(true_chars_percent, label='percentage of correct characters', color='orange')

  # generate x coordinates (indices of the array)
  x_coords = range(len(confus))

  # compute y coordinates (confus values divided by 3)
  y_coords = [num / 3 for num in confus]

  # define colors and labels for each value
  colors = {0: 'purple', 1: 'red', 2: 'grey', 3: 'green'}
  labels = {0: 'False Negaties', 1: 'False Positives', 2: 'True Negatives', 3: 'True Positives'}
  plotted_labels = set()

  # each point with a different color
  for x, y in zip(x_coords, y_coords):
      value = int(y * 3)
      if value not in plotted_labels:
          plt.scatter(x, y, color=colors[value], label=labels[value])
          plotted_labels.add(value)
      else:
          plt.scatter(x, y, color=colors[value])

  plt.xlabel('iteration')
  plt.ylabel('percentage')
  plt.legend()

  # save the plot
  plt.savefig(f'{my_path}/plots/markov_chain_test_conf{_}.png')
  plt.show()

#### Plot used in Figure 28

In [ ]:
for _ in range(5):
  text = get_testing_texts(1)
  key = create_permutation_key(27, 27)

  ciphertext = substitute_encrypt(text, key)
  likelihoods, true_chars_percent, likelihoods_second_func, confus = monte_carlo_algorithm(ciphertext, 3001, rnn_estimate_function, original_text=text)

  plt.figure(figsize=(10, 6))

  # plot the likelihood and correct characters % using the index as the x-value
  plt.plot(likelihoods, label='RNN estimate', color='blue')
  plt.plot(true_chars_percent, label='percentage of correct characters', color='orange')

  # generate x coordinates (indices of the array)
  x_coords = range(len(confus))

  # compute y coordinates (confus values divided by 3)
  y_coords = [num / 3 for num in confus]

  # define colors and labels for each value
  colors = {0: 'purple', 1: 'red', 2: 'grey', 3: 'green'}
  labels = {0: 'False Negaties', 1: 'False Positives', 2: 'True Negatives', 3: 'True Positives'}
  plotted_labels = set()

  # plot each point with a different color
  for x, y in zip(x_coords, y_coords):
      value = int(y * 3)
      if value not in plotted_labels:
          plt.scatter(x, y, color=colors[value], label=labels[value])
          plotted_labels.add(value)
      else:
          plt.scatter(x, y, color=colors[value])

  plt.xlabel('iteration')
  plt.ylabel('percentage')
  plt.legend()

  # save the plot
  plt.savefig(f'{my_path}/plots/rnn_test_conf{_}.png')


### Data pipelines
downloading data in .csv format

[Wikipedia Webscrapping]() the most topics, use of webscrapping



In [ ]:
def fetch_random_wikipedia_articles(language='en', num_articles=50):
    # Wikipedia API endpoint
    url = f'https://{language}.wikipedia.org/w/api.php'

    # Parameters for the API request to get random articles
    params = {
        'action': 'query',
        'format': 'json',
        'list': 'random',  # random articles
        'rnnamespace': 0,  # namespace 0 is for main articles (not user pages)
        'rnlimit': num_articles  # number of random articles to return
    }

    response = requests.get(url, params=params)
    data = response.json()

    articles_content = []

    for article in data['query']['random']:
        page_title = article['title']

        # Parameters for the API request to get the content of the article
        content_params = {
            'action': 'query',
            'format': 'json',
            'titles': page_title,
            'prop': 'extracts',
            'explaintext': True,  # get the content as plaintext
        }

        content_response = requests.get(url, params=content_params)
        content_data = content_response.json()

        # Extract page id to access the content
        page_id = next(iter(content_data['query']['pages']))
        content = content_data['query']['pages'][page_id]['extract']

        articles_content.append(content)

    return ' '.join(articles_content)


In [ ]:
def split_string_fixed_length(input_string, length):
    # Split the string into chunks of the specified length
    # The remainder that doesn't fit into the length is cut off
    chunks = [input_string[i:i+length] for i in range(0, len(input_string), length)]

    if len(chunks[-1]) < length:
        chunks = chunks[:-1]

    return chunks

In [ ]:
def webscrapping_wiki(length=128, n_datapoints=250000, n_swapped_elements=0, language='en'):
    filename = f'{my_path}/wikidata/512_data.csv'

    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)

        counter = 0
        while counter < n_datapoints:
            random_text = fetch_random_wikipedia_articles(language, 100)  # random texts from Wikipedia
            try:
                formatted_random_text = format_text(random_text)  # formatting to A-Z & _
            except AssertionError:  # sometimes empty string is fetched, then formatting fails
                continue
            else:
                text_chunked = split_string_fixed_length(formatted_random_text, length)  # creating chunks
                if len(text_chunked) + counter > n_datapoints:  # ensuring the desired length of the dataset is not exceeded
                    text_chunked = text_chunked[:n_datapoints - counter]
                for text in text_chunked:
                    new_key = create_permutation_key(n_swapped_elements, 27)
                    encrypted_text = substitute_encrypt(text, new_key)
                    encoding = sum([1 if text[i] == encrypted_text[i] else 0 for i in range(len(text)) ]) / len(text)
                    writer.writerow([encrypted_text, encoding])
                counter += len(text_chunked)
        print(f'encoding {counter} texts done')

#### Webscrapping call
scrapping data for 50000 texts of length 256 from wikipedia takes around 25 minutes

In [ ]:
# need to be run only once
# webscrapping_wiki(512, 38344, 0)

In [ ]:
def combine_csv_files(file1, file2, output_file):
    # Read the CSV files
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Concatenate the dataframes
    combined_df = pd.concat([df1, df2], ignore_index=True)

    # Write the combined dataframe to a new CSV file
    combined_df.to_csv(output_file, index=False)

# Example usage
file1 = f'{my_path}/wikidata/new_128_data.csv'
file2 = f'{my_path}/wikidata/128_250000_encrypted.csv'
output_file = f'{my_path}/wikidata/model6.csv'
combine_csv_files(file1, file2, output_file)

In [ ]:
# File path to the CSV file
file_path = f'{my_path}/wikidata/512_data.csv'

# Define column defaults: first column is a string, second column is a float
column_defaults = [tf.string, tf.float32]

# Create a CSV dataset
dataset = tf.data.experimental.CsvDataset(
    file_path,
    record_defaults=column_defaults,
    header=True,
)

# Convert the dataset to a list of tuples for iteration
data_list = list(dataset.as_numpy_iterator())



In [ ]:
# Iterate through the dataset and perform manual changes
altered_data = []
for index, (text, encoding_level) in enumerate(data_list):
    text = text.decode('utf-8')
    new_key = create_permutation_key(index % 28, 27)
    encrypted_text = substitute_encrypt(text, new_key)
    encoding = sum([1 if text[i] == encrypted_text[i] else 0 for i in range(len(text)) ]) / len(text)

    # Append the altered data to the list
    altered_data.append((encrypted_text, encoding))

# Convert the list of tuples to a DataFrame
df = pd.DataFrame(altered_data, columns=['Text', 'EncodingLevel'])

# Save the altered DataFrame to a new CSV file
altered_file_path = f'{my_path}/wikidata/512_data_encoded.csv'
df.to_csv(altered_file_path, index=False)

### Line profiler
checking the performance of the code - must be run within jupyter-notebook environment

In [ ]:
%load_ext line_profiler

#### format text function
most expensive is the unidecode call - which is necessery to convert foreign characters into english ones

checking, if the input text is already formatted using RegEx (also costly) is important to prevent sneaky bugs (HELLO_WORLD would turn into HELLOWORLD)

In [ ]:
%lprun -f format_text format_text(format_test_text)

#### text to vector
most expensive is formatting the text beforehand (unidecode)

In [ ]:
test_vector = text_to_vector(format_test_text)
%lprun -f text_to_vector text_to_vector(format_test_text)

In [ ]:
%lprun -f vector_to_text vector_to_text(test_vector)

#### Substitute en-/decryption
again - most costly is the subfunction text_to_vector

In [ ]:
test_key = np.random.permutation(27)
%lprun -f substitute_encrypt substitute_encrypt(format_test_text, test_key)

In [ ]:
%lprun -f substitute_decrypt substitute_decrypt(format_test_text, test_key)

#### Creating permutation key
given percentage of swapped elements and number of elements in the vector
most costly operation is using RNG to choose one of the remaining indices --> alternative approach would be to use the first index possible - but this would return the same permutation key everytime.

In [ ]:
%lprun -f create_permutation_key create_permutation_key(10, n_elements=27)